# Оптимизация основого кода + импорт текста из GitHub'а

In [2]:
import os
import re
import glob
import time
import math
import pandas as pd
from tqdm import tqdm
from docx import Document
from pdfminer.high_level import extract_text
import spacy
import requests

## Кэширование моделей spaCy

In [4]:
# Кэшируем модели spaCy для русского и английского языков один раз
nlp_ru = spacy.load("ru_core_news_sm")
try:
    nlp_en = spacy.load("en_core_web_sm")
except Exception as e:
    print("Ошибка загрузки английской модели spaCy:", e)
    nlp_en = None

## Функция чтения резюме из файла

In [6]:
def read_resume_from_file(file_path):
    """
    Читает резюме из файла (поддерживаются форматы TXT, DOCX, PDF).
    """
    if file_path.lower().endswith('.txt'):
        try:
            with open(file_path, encoding='utf-8') as f:
                text = f.read()
            return text
        except Exception as e:
            print(f"Ошибка при чтении TXT файла {file_path}: {e}")
            return None
    elif file_path.lower().endswith('.docx'):
        try:
            doc = Document(file_path)
            text = "\n".join([para.text for para in doc.paragraphs])
            return text
        except Exception as e:
            print(f"Ошибка при чтении DOCX файла {file_path}: {e}")
            return None
    elif file_path.lower().endswith('.pdf'):
        try:
            text = extract_text(file_path)
            return text
        except Exception as e:
            print(f"Ошибка при чтении PDF файла {file_path}: {e}")
            return None
    else:
        print(f"Неподдерживаемый формат файла: {file_path}")
        return None

## Словарь компетенций

In [8]:
competencies_dict = {
    # ОБЩИЕ КОМПЕТЕНЦИИ
    "Определения, история развития и главные тренды ИИ": [
        "определение ии", "история ии", "развитие ии", "главный тренд ии", "концепция ии",
        "новый тренд ии", "искусственный интеллект", "ai definition", "history ai", "ai trend", "artificial intelligence",
        "внедрение ии", "интеграция ии-решений", "чат-боты", "осваивать ии", "git", "jupyter notebook", "json",
        "postman", "api", "xml", "xsd", "прототипирование", "анализ данных", "аналитика", "numpy", "selenium",
        "работа с ии", "digital economy", "ai-driven рост", "нейронная сеть", "terra ai", "ml", "модель ии",
        "machine learning", "deep learning", "ai models", "llm", "gen-ai"
    ],
    "Процесс, стадии и методологии разработки решений на основе ии (docker, linux/bash, git)": [
        "процесс разработка решение", "стадия разработка", "методология разработка",
        "docker", "linux", "bash", "git", "pipeline", "ci/cd", "development process", "development stage",
        "continuous integration", "continuous delivery", "dax", "gitlab", "github", "bitbucket",
        "python", "разработка и поддержка etl пайплайн", "luigi", "airflow", "ai менеджмент",
        "разработка ai-продуктов", "rag", "ai-инструменты", "replit", "chatgpt", "api", "google cloud",
        "app engine", "unix", "постановка задач", "инструкции", "sql", "дашборды", "agile", "jira",
        "интеграция api", "работа с базой данных", "автоматизация", "fastapi", "flask", "обучение модель",
        "mvp", "продукт", "запуск", "roadmap", "ml", "оптимизация", "анализ", "метрика", "a/b тест",
        "websocket", "aws", "quantization", "inference", "deployment", "multithreading", "docker compose",
        "cvat", "minio-хранилища", "redmine", "python backend"
    ],
    "Статистические методы и первичный анализ данных": [
        "статистический метод", "первичный анализ данные", "описательный статистика", "регрессия",
        "корреляция", "дисперсионный анализ", "гипотеза проверка",
        "statistical method", "primary data analysis", "descriptive statistic", "regression", "correlation",
        "variance analysis", "hypothesis test", "probability", "statistics", "очистка", "выгрузка", "etl",
        "сырой", "elt", "обработка", "предобработка", "нормализация", "обогащение", "дистрибуция",
        "pandas", "подготовка", "замена текста", "агрегация", "существование данных", "поиск данных", "дедупликация",
        "статистический анализ", "аналитические исследования", "прогнозирование", "сегментация", "a/b тесты",
        "математика", "аналитика", "анализ бизнес-процессов", "анализ данных", "аналитика продаж",
        "анализ бизнес-показателей", "работа с датасетами", "numpy", "scikit-learn", "sklearn", "регрессионный анализ",
        "математическая статистика", "excel", "формула", "сводная таблица", "abc-анализ", "rfm-анализ",
        "когортный анализ", "сезонность", "pca", "data cleaning"
    ],
    "Промпт-инжиниринг": [
        "промпт инженеринг", "prompt engineering", "генерация промпт", "оптимизация промпт", 
        "prompt design", "chain of thought", "промпт-инженер", "chatgpt", "elevenlabs", "colossyan",
        "разработка ai чат-ботов", "prompt tuning", "llama 3", "llama factory", "настройка промптов",
        "prompt optimization", "prompt crafting", "подбор промптов", "prompt modification"
    ],
    "Инструменты CitizenDS": [
        "citizends", "citizen ds", "инструмент citizends", "citizends platform",
        "low code", "no code", "power bi", "power query", "tableau", "google sheets",
        "citizends analytics", "citizends automation", "citizends integration", "citizends dashboard"
    ],
    "Оценка качества работы методов ИИ": [
        "оценка качество", "качество метод ии", "валидация ии", "evaluation method ai", "тестирование ии",
        "метрика точность", "precision", "recall", "f1", "quality model",
        "статистический анализ", "анализ данных", "предсказательная модель", "sklearn", "метрика",
        "анализ результат", "ner", "классификация", "рекомендация", "ml-рекомендации", "accuracy",
        "quantization", "model orchestration", "оценка моделей llm", "сравнение моделей", "автотесты"
    ],
    "Языки программирования и библиотеки (python, c++)": [
        "язык программирование", "библиотека python", "библиотека c++", "python", "c++", 
        "programming", "library", "объектно ориентированный", "oop", "java", "script",
        "c", "c+", "c#", "pandas", "javascript", "reactjs", "apache spark", "1с", "vbs", "golang",
        "scala", "pathlib", "exchangelib", "rust", "numpy", "requests", "tkinte", "jupyter notebook",
        "selenium", "pytorch", "tensorflow", "matlab", "r", "sklearn", "keras", "typescript", "bash",
        "html5", "css3", "vue", "react", "sass", "mobx"
    ],
    "Этика ИИ": [
        "этика ии", "ethical ai", "ai ethic", "ответственный ии", "responsible ai", "privacy", "fairness",
        "этические нормы", "transparency", "accountability", "ai governance"
    ],
    "Безопасность ИИ": [
        "безопасность ии", "кибербезопасность", "cybersecurity", "data security", "защита информация",
        "information protection", "adversarial", "robustness"
    ],
    "Цифровые двойники": [
        "цифровой двойник", "digital twin", "виртуальный двойник", "simulation", "цифровое моделирование",
        "digital replica", "виртуальный клон"
    ],
    
    # АЛГОРИТМЫ И МЕТОДЫ ИИ
    "Методы машинного обучения": [
        "метод машинный обучение", "machine learning method", "ml method", "алгоритм ml", "ml algorithm",
        "кластерная модель", "прогнозирование", "ltv", "retention rate", "python", "аналитика", "математика",
        "ии-решения", "чат-боты", "анализ данных", "numpy", "jupyter notebook", "random forest",
        "scikit-learn", "xgboost", "основы машинного обучения", "введение в машинное обучение",
        "обучение предсказательной модели", "sklearn", "машинное обучение", "classification", "регрессия",
        "обучение", "fine-tuning", "ensemble learning", "semi supervised learning", "transformers", "cnns", "vae", "gans",
        "llms", "generative models", "алгоритмы и структуры данных"
    ],
    "Методы оптимизации": [
        "метод оптимизация", "optimizer", "optimization", "градиентный спуск", "gradient descent",
        "stochastic gradient descent", "hyperparameter tuning", "adam", "adagrad", "rmsprop",
        "форматирование", "нормализация", "пайплайн", "методология", "рефакторинг", "улучшение",
        "генетический алгоритм", "оптимальный выбор", "оптимизация процессов", "accuracy enhancement",
        "математическое программирование", "улучшение алгоритмов", "прогнозирование", "оптимизация бизнес-процессов",
        "устранение узких мест", "бизнес-анализ", "bpmn", "idef0", "ускорение принятия решений",
        "автоматизация бизнес-процессов", "создание эффективных пайплайнов для обработки больших объёмов данных",
        "luigi", "airflow"
    ],
    "Информационный поиск": [
        "информационный поиск", "information retrieval", "search", "retrieval", "поисковый", "indexing",
        "скрейпинг", "scrappy", "загрузка", "парсинг", "выгрузка", "сбор данных", "разметка", "интеграция",
        "bitrix", "1с", "api", "источник", "json", "csv", "xlsx", "sql таблицы", "анализ конкурентов",
        "data mining", "анализ google maps", "statista", "eurostat", "системный анализ",
        "исследования и разработки", "поиск токсичность", "поиск ner", "мониторинг источников"
    ],
    "Рекомендательные системы": [
        "рекомендательный система", "recommendation system", "collaborative filtering", "content based",
        "matrix factorization", "система рекомендаций контента", "сегментация клиентов", "предсказание покупки",
        "улучшение динамики бронирований", "ретаргетинг", "предсказание поведение", "рекомендации"
    ],
    "Анализ изображений и видео": [
        "анализ изображение", "анализ видео", "image analysis", "video analysis", "computer vision", "cv",
        "yolov4", "yolov5", "cv2", "object detection", "openface", "inpaint", "lama", "image processing",
        "3d pose estimation", "segmentation", "unet", "pspnet", "gan pix2pix", "семантическая сегментация",
        "cnns", "keras", "tensorflow", "opencv"
    ],
    "Анализ естественного языка": [
        "анализ естественный язык", "nlp", "natural language processing", "text analysis", "language understanding",
        "ner", "transformers", "huggingface", "nltk", "textblob", "перевод", "арабский", "локализация",
        "разметка текст", "automated assessment", "instance segmentation", "image annotation", "foam segmentation",
        "обучение llm", "llms", "prompt tuning", "обучение на пользовательских данных"
    ],
    "Основы глубокого обучения": [
        "основа глубокий обучение", "deep learning basic", "нейронный сеть основа", "basic neural network",
        "keras", "pytorch", "deep learning", "fine-tuning", "ml", "tensorflow", "matlab", "r", "sklearn",
        "numpy", "jupyter notebook"
    ],
    "Глубокое обучение для анализа и генерации изображений, видео": [
        "глубокий обучение", "анализ изображение", "генерация изображение", "видео анализ", "image generation",
        "video generation", "gan", "cnn", "dnn", "inpaint", "lama", "deep learning видео", "instance segmentation",
        "image annotation", "foam segmentation", "llms", "transformers", "llama 3", "prompt tuning", "llama factory",
        "d2go", "unet", "yolo"
    ],
    "Глубокое обучение для анализа и генерации естественного языка": [
        "глубокий обучение", "анализ естественный язык", "генерация текст", "transformer", "rnn",
        "language model", "text generation", "transformers", "huggingface", "automated feedback",
        "ai-led interviews", "llm", "langchain", "генерация и анализ текста", "nlp"
    ],
    "Обучение с подкреплением и глубокое обучение с подкреплением": [
        "обучение с подкрепление", "reinforcement learning", "deep reinforcement learning", "rl",
        "policy gradient", "q learning", "dqn"
    ],
    "Гибридные модели и PIML": [
        "гибридный модель", "piml", "ensemble", "интеграция модель", "physics informed", "hybrid model",
        "model fusion", "смешанные модели", "интеграция моделей", "гибридная система"
    ],
    "Анализ геоданных": [
        "анализ геоданные", "географический данные", "gis", "пространственный анализ", "spatial analysis",
        "geopandas", "kepler", "shapely", "h3", "geopy", "города", "сравнение городов", "геоанализ",
        "анализ google maps", "определение стратегий по регионам"
    ],
    "Массово параллельные вычисления для ускорения машинного обучения (gpu)": [
        "массово параллельный вычисление", "ускорение обучение", "gpu", "cuda", "opencl", "gpu parallel",
        "keras", "pytorch", "tensorrt", "quantization", "onnx", "luigi", "etl-процессы", "docker"
    ],
    "Работа с распределенной кластерной системой": [
        "работа распределенный кластерный система", "cluster computing", "distributed system", "mapreduce",
        "aws", "greenplum", "clickhouse", "кластер", "распределенный", "high", "load", "mwaa", "dagster",
        "репликация данных", "k8s", "tensorrt", "google cloud", "bigquery", "airflow", "unix"
    ],
    "Машинное обучение на больших данных": [
        "машинный обучение", "больший данные", "machine learning", "big data", "scalable ml",
        "скрапинг", "semi supervised learning", "inference in production", "llms",
        "обучение и инференс моделей", "kubernetes", "bigquery", "google cloud", "gps", "большие данные",
        "sklearn", "pandas", "работа с большим объёмом информации", "предсказательная модель", "bi аналитика",
        "power bi", "postgres", "mysql", "qlikview", "tableau", "анализ данных", "большие объёмы информации",
        "аналитика", "jupyter", "python", "анализ больших объемов информации", "statista", "eurostat"
    ],
    "Потоковая обработка данных (data streaming, event processing)": [
        "потоковый обработка данные", "data streaming", "event processing", "stream processing", "realtime data",
        "clickhouse", "greenplum", "elt", "etl", "база", "хранилище", "rabbit", "очередь", "пайплайн",
        "большие данные", "big", "dbt", "биллинг", "распределенный", "teradata", "streaming", "процессинг",
        "поток", "потоковый", "конвеер", "пользовательский поток", "event", "интервал скрапинга",
        "поток данных", "streaming communication", "websocket", "api", "airflow", "интеграция api"
    ],
    "Графовые нейросети": [
        "графовый нейросеть", "graph neural network", "gnn", "graph embedding", "node classification",
        "graph convolution", "graph attention", "gat", "graph pooling"
    ],
    
    # РАБОТА С ДАННЫМИ
    "SQL базы данных (greenplum, postgres, oracle)": [
        "sql", "greenplum", "postgres", "oracle", "relational database", "sql база данные",
        "ms", "postgre", "postgresql", "db", "clickhouse", "t", "transact", "mysql", "mssql server"
    ],
    "NoSQL базы данных (cassandra, mongodb, elasticsearch, neo4j, hbase)": [
        "nosql", "cassandra", "mongodb", "couchdb", "redis", "firebase", "dynamodb", "neo4j", "elasticsearch",
        "document store", "key-value", "wide-column"
    ],
    "Массово параллельная обработка и анализ данных": [
        "массово параллельный обработка", "анализ данные", "massively parallel", "big data processing", "distributed analysis",
        "анализ", "даные", "pandas", "airflow", "nifi", "rabbit", "коннектор", "витрина", "метрика", "база", "excel",
        "xlsx", "xls", "csv", "lxml", "xml", "результат", "отчет", "динамика", "аналитика", "пайплайн", "bi", "power",
        "навигация", "bitrix", "битрикс", "1с", "отчетность", "обогащение", "нормализация", "аналитик", "исследовательский",
        "дэшборд", "графический", "анализировать", "ad-hoc", "сводный", "таблица", "query", "datalens", "superset",
        "datastudio", "юнит", "оркестровка", "экспертиза",
        "figma", "tilda", "тильда", "ux/ui", "adobe photoshop", "дизайн", "photoshop"
    ],
    "Hadoop, spark, hive": [
        "hadoop", "spark", "hive", "yarn", "mapreduce", "apache hadoop", "apache spark", "apache hive", "hiveql"
    ],
    "Шины данных (kafka)": [
        "kafka", "шина данные", "data bus", "message queue", "kafka streams", "event streaming", "distributed messaging"
    ],
    "Качество и предобработка данных, подходы и инструменты": [
        "качество данные", "предобработка данные", "data preprocessing", "data quality", "etl", "cleaning",
        "сырой", "pandas", "нормализация", "обновление", "приведение", "единый формат", "обогащение", "дистрибуция",
        "jupyter", "сегментация", "сегментирование", "регулярный", "шаблон", "поиск", "анализ", "замена", "dag",
        "агрегация", "запись", "чтение", "обработка", "оркестровка", "проверка", "удаление", "дедупликация"
    ],
    "Графы знаний и онтологии": [
        "граф знание", "онтология", "knowledge graph", "ontology", "semantic network", "reasoning",
        "ontological modeling", "semantic web", "ontology engineering"
    ]
}

## Функции для обработки текста и аннотации

In [10]:
def preprocess_text(text):
    """
    Очищает текст: удаляет лишние пробелы, переносы строк и переводит в нижний регистр.
    """
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text.lower()

def detect_language(text):
    """
    Определяет язык текста: если доля кириллицы > 20% – возвращает "ru", иначе "en".
    """
    cyrillic = sum(1 for ch in text if 'а' <= ch <= 'я' or 'А' <= ch <= 'Я')
    return "ru" if len(text) > 0 and (cyrillic / len(text)) > 0.2 else "en"

def tokenize_and_lemmatize(text, lang):
    """
    Выполняет токенизацию и лемматизацию текста с использованием кэшированных моделей spaCy.
    """
    if lang == "ru":
        doc = nlp_ru(text)
    else:
        if nlp_en is None:
            return [], []
        doc = nlp_en(text)
    tokens = [token.text for token in doc]
    lemmas = [token.lemma_ for token in doc]
    return tokens, lemmas

def full_lemmatize_text(text, lang):
    """
    Лемматизирует весь текст и возвращает строку лемм, объединённых пробелами.
    """
    processed = preprocess_text(text)
    _, lemmas = tokenize_and_lemmatize(processed, lang)
    return " ".join(lemmas)

def lemmatize_phrase(phrase, lang):
    """
    Лемматизирует отдельное ключевое словосочетание.
    """
    phrase = phrase.strip().lower()
    _, lemmas = tokenize_and_lemmatize(phrase, lang)
    return " ".join(lemmas)

def annotate_combined_text(combined_lem_text):
    """
    Выполняет аннотацию по уже лемматизированному объединённому тексту.
    Для каждой компетенции проверяет наличие лемматизированных ключевых фраз на русском или английском.
    """
    annotations = {}
    for competence, keywords in competencies_dict.items():
        if keywords is None:
            continue
        found = False
        for kw in keywords:
            # Проверяем и с русской, и с английской лемматизированной формой ключевой фразы
            kw_ru = lemmatize_phrase(kw, "ru")
            kw_en = lemmatize_phrase(kw, "en")
            if kw_ru in combined_lem_text or kw_en in combined_lem_text:
                found = True
                break
        annotations[competence] = 1 if found else 0
    return annotations

## Функции для работы с GitHub

In [12]:
def extract_github_links_from_text(text):
    """
    Извлекает GitHub-ссылки из текста.
    """
    github_pattern = r"https?://(?:www\.)?github\.com/[A-Za-z0-9_-]+"
    return list(set(re.findall(github_pattern, text)))

def get_github_username_from_url(url):
    """
    Извлекает имя пользователя GitHub из URL.
    """
    match = re.match(r"https?://github\.com/([^/]+)", url)
    return match.group(1) if match else None

def get_repos(username):
    """
    Получает список репозиториев пользователя GitHub через API.
    """
    url = f"https://api.github.com/users/{username}/repos"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Ошибка при получении репозиториев для {username}")
        return []

def get_readme_text(username, repo):
    """
    Получает текст README для репозитория через GitHub API.
    """
    url = f"https://api.github.com/repos/{username}/{repo}/readme"
    headers = {'Accept': 'application/vnd.github.v3.raw'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    return ""

def collect_github_text(github_url):
    """
    По заданной GitHub-ссылке извлекает имя пользователя, затем получает тексты README
    всех репозиториев этого пользователя и возвращает их объединённый текст.
    """
    username = get_github_username_from_url(github_url)
    if not username:
        return ""
    repos = get_repos(username)
    all_texts = []
    for repo in repos:
        repo_name = repo.get("name")
        readme = get_readme_text(username, repo_name)
        if readme:
            all_texts.append(readme)
    return " ".join(all_texts)

## Формирование датасета с интеграцией GitHub-текста и выводом каждые 10% файлов

In [14]:
# Запрашиваем у пользователя путь к папке с резюме
folder = input("Введите путь к папке с резюме: ").strip()

# Рекурсивный поиск всех файлов в указанной папке и поддиректориях
files = glob.glob(os.path.join(folder, '**', '*.*'), recursive=True)
print(f"Найдено файлов: {len(files)}")

dataset_list = []
# Определяем порядок компетенций (ключи, для которых значение не None)
# Здесь мы используем ключи из обновлённого словаря, их общее число считается 36 (если так задумано)
competency_list = list(competencies_dict.keys())

total_files = len(files)
# Вычисляем пороговые значения каждые 10%
thresholds = [math.floor(total_files * p / 100) for p in range(10, 101, 10)]
thresholds = sorted(set([t for t in thresholds if t > 0]))

start_time = time.time()  # запуск секундомера

for i, file in enumerate(files):
    text = read_resume_from_file(file)
    if text:
        # Получаем очищенный текст резюме (не лемматизированный)
        resume_clean_text = preprocess_text(text)
        
        # Извлекаем GitHub-ссылки из исходного текста резюме
        gh_links = extract_github_links_from_text(text)
        gh_text_combined_raw = ""
        if gh_links:
            for link in gh_links:
                raw_gh_text = collect_github_text(link)
                if raw_gh_text:
                    # Применяем очистку к GitHub-тексту (без лемматизации)
                    gh_clean_text = preprocess_text(raw_gh_text)
                    gh_text_combined_raw += " " + gh_clean_text
        
        # Формируем комбинированный очищенный текст: резюме + GitHub-текст (если есть)
        if gh_text_combined_raw.strip():
            combined_raw_text = resume_clean_text + " " + gh_text_combined_raw
        else:
            combined_raw_text = resume_clean_text
        
        # Для аннотации создаём лемматизированную версию объединённого текста
        combined_lang = detect_language(combined_raw_text)
        combined_lem_text = full_lemmatize_text(combined_raw_text, combined_lang)
        annotation_dict = annotate_combined_text(combined_lem_text)
        # Генерируем вектор меток для каждой компетенции
        labels = [annotation_dict.get(comp, 0) for comp in competency_list]
        
        dataset_list.append({
            "filename": os.path.basename(file),
            "text": combined_raw_text,
            "labels": labels
        })
    
    # Выводим промежуточный результат каждые 10% от общего числа файлов
    if (i + 1) in thresholds:
        percentage = (i + 1) / total_files * 100
        print(f"Обработано {i + 1} файлов из {total_files} ({percentage:.1f}%)")

elapsed = time.time() - start_time  # вычисление времени выполнения
print(f"\nОбработка завершена за {elapsed:.2f} секунд ({elapsed/60:.2f} минут)")
print(f"Сформировано записей датасета: {len(dataset_list)}")

df = pd.DataFrame(dataset_list)
print("\nПервые 25 записей датасета:")
print(df.head(25))

df.to_csv("resume_dataset.csv", index=False, encoding="utf-8-sig")
print("\nДатасет сохранён в файл 'resume_dataset.csv'")

Введите путь к папке с резюме:  C:\Users\HP\Desktop\Хакатоны\ГазпромНефть_сервис_для_HR\Газпром_кейс_все_резюме


CropBox missing from /Page, defaulting to MediaBox


Найдено файлов: 733


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Ошибка при получении репозиториев для vlbudaeva


...

Ошибка при получении репозиториев для Alica032


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Ошибка при получении репозиториев для oldrzym


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Обработано 733 файлов из 733 (100.0%)

Обработка завершена за 3419.24 секунд (56.99 минут)
Сформировано записей датасета: 727

Первые 25 записей датасета:
                                             filename  \
0                       (Разработчик_ML Engineer).pdf   
1   adham-by-ag-s8-data-analytic-data-analyst-seni...   
2   adylhan-x7q8v0w1dy-analitik-rossiya-kazahstan.pdf   
3   aiperi-labi6f9-2-devops-inzhener-data-inzhener...   
4   ajbek-5i-8-htrq-devops-inzhener-sistemnyj-anal...   
5          albert-v-ejvhet-analitik-rossiya-kazan.pdf   
6   aleksandr-17gnzq-ox9-quantitative-developer-gr...   
7   aleksandr-1o-gf-3ql-razrabotchik-1s-proekt-men...   
8   aleksandr-62boogvrdq-data-scientist-yerevan-ar...   
9   aleksandr-7haueass-s-sistemnyj-analitik-udalyo...   
10  aleksandr-aeqi-l3qir-analitik-dannyh-rossiya-m...   
11  aleksandr-br-kj-q-ck-data-analyst-vengriya-ser...   
12  aleksandr-elmup-1p-produktovyj-analitik-moskva...   
13  aleksandr-hgceow-dpr-sistemnyj-analitik-tai